In [1]:
import numpy as np
import cv2
import sys
import glob
from matplotlib import pyplot as plt
import time

In [2]:
# training data
dataset_path = "dataset/image/design1/*.jpg"
files = glob.glob(dataset_path) 

In [18]:
# test data
# test_path = "dataset/video/*.avi"
test_path = "dataset/video/*.mkv"
test_files = glob.glob(test_path) 

In [19]:
test_files

['dataset/video/สอนขับรถออกถนนใหญ่ และกฏจราจร - ครูเพ็ญ สอนขับรถยนต์.mkv']

In [20]:
ksize = 7

In [21]:
# noise removal
def remove_noise(image, k):
    return cv2.GaussianBlur(image,(k,k),0)
    # return cv2.medianBlur(image,k)

#dilation
def dilate(image, cnt_dilate):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = cnt_dilate)
    
#erosion
def erode(image, cnt_erode):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = cnt_erode)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 255)


In [ ]:
def get_mark(img):
    # img = cv2.imread(dir)
    print( 'img',img.shape )
    # resize
    h, w, _= img.shape
    if h > w:
        img = cv2.resize(img,(360, round(360 * h / w) ))        
    else:
        img = cv2.resize(img,(round(360 * w / h),360))
    print( 'img-resize',img.shape )
    # rgb for present
    img_org = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    # gray
    img_gray = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
    img_gray = remove_noise(img_gray, 3)
    
    img_candy = canny(img_gray)

    # Find all contours
    contours, hierarchy = cv2.findContours(img_candy,
                                          cv2.RETR_LIST,
                                          cv2.CHAIN_APPROX_NONE)

    contours_mark = []
    max_area = 0
    max_length = 0
    img_box = img.copy()
    for c in contours:
        if cv2.contourArea(c) > max_area:
            max_area = cv2.contourArea(c)
        if cv2.arcLength(c,False) > max_length:
            max_length = cv2.arcLength(c,False)
        if (cv2.contourArea(c) > 150 ) or (cv2.arcLength(c,False) > 150 ):
            contours_mark.append(c)
            x,y,w,h = cv2.boundingRect(c)
            print('w,h',w,h, w/h)
            is_square = False
            
            if w/h > 0.8 and w/h < 1.2:
                is_square = True
            if is_square:
                cv2.rectangle(img_box,(x,y),(x+w,y+h),(0,0,255),2)
        # print(cv2.contourArea(c),cv2.arcLength(c,False))
    print('max', max_area, max_length)

    img_box = cv2.drawContours( img_box.copy(),contours_mark,-1, 
                  (0,255,0), 
                  thickness=1)
                  
    # img_object = mark_object
    # title
    # titles = [ 'img_org',  'img_gray','img_candy','mark_object','img_box']
    # images = [ img_org,  img_gray ,img_candy,mark_object,img_box]

    # # Show all resultant images using pyplot
    # fig = plt.figure(figsize=(15,8))
    # fig.canvas.set_window_title('OpenCV')
    # for i in range(len(titles)):
    #     plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
    #     plt.title(titles[i])
    #     plt.xticks([]),plt.yticks([])
    return img_box

In [26]:
dir = test_files[0]
capture = cv2.VideoCapture(dir)

# Initialize the timer
n_frames = 0              # the number of frames
start_time = time.time()  # the start time

delay = 10       # delay in millisecond for better observation regarding tracking results
while True:
    retval,img = capture.read()
    img = get_mark(img)
    print(img.shape)
    # Draw a text label specifying the current fps
    n_frames += 1
    total_time = time.time() - start_time
    fps = n_frames / total_time
    h, w, c = img.shape
    caption_h = 40
    frame_fps = np.zeros( (h+caption_h,w,c), dtype=np.uint8 )
    frame_fps[ caption_h:, ... ] = img
    cv2.putText( frame_fps,                  # image to draw text
                 f'fps={fps:.2f}',           # text to be written
                 (10,25),                    # position coordinates of the text
                 cv2.FONT_HERSHEY_COMPLEX,   # font type
                 0.75,                       # font scale
                 (255,255,255),              # font BGR color
                 1,                          # font thickness
                 cv2.LINE_AA )

    cv2.imshow('MultiTracker: ',frame_fps)

    # Wait 1 millisecond for any key press        
    if (cv2.waitKey(max(1,delay))== 27):       # press ESC to quit
        break
cv2.destroyAllWindows()
if capture.isOpened():
    capture.release()

img (1080, 1920, 3)
img-resize (360, 640, 3)
w,h 197 29 6.793103448275862
w,h 202 12 16.833333333333332
w,h 105 21 5.0
w,h 120 21 5.714285714285714
w,h 58 57 1.0175438596491229
w,h 113 23 4.913043478260869
w,h 64 14 4.571428571428571
w,h 109 25 4.36
w,h 448 118 3.7966101694915255
w,h 337 99 3.404040404040404
w,h 103 10 10.3
w,h 108 2 54.0
w,h 31 19 1.631578947368421
w,h 58 31 1.8709677419354838
w,h 64 21 3.0476190476190474
w,h 80 27 2.962962962962963
w,h 85 29 2.9310344827586206
w,h 43 38 1.131578947368421
w,h 46 22 2.090909090909091
w,h 55 20 2.75
w,h 32 25 1.28
w,h 27 20 1.35
w,h 39 40 0.975
w,h 87 8 10.875
w,h 36 27 1.3333333333333333
w,h 19 56 0.3392857142857143
w,h 56 51 1.0980392156862746
w,h 44 20 2.2
w,h 58 37 1.5675675675675675
w,h 22 37 0.5945945945945946
w,h 18 80 0.225
w,h 76 83 0.9156626506024096
w,h 86 41 2.097560975609756
w,h 44 90 0.4888888888888889
w,h 40 33 1.2121212121212122
w,h 59 104 0.5673076923076923
w,h 49 60 0.8166666666666667
w,h 37 154 0.24025974025974026
w,h